In [2]:
# install necesary packages
%pip install import-ipynb

%pip install pandas
%pip install numpy
%pip install sklearn
%pip install langchain langchain-community openai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 's

In [3]:
# Lang chain imports
from langchain.chat_models import ChatOpenAI  
from langchain.schema import HumanMessage, SystemMessage
from dotenv import load_dotenv

# Get API KEY
import os

load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("OpenAI API Key not found")

# Import libraries
import pandas as pd
import numpy as np

In [4]:
# Import features (TODO: Add features as we develop)
import import_ipynb
# from features.summaries import get_summaries
# from features.outliers import detect_outliers

In [5]:
def main(user_query, df):
    
    # List features for agent 
    features = """
    Available features:
    - get_summaries(): summaries 
    - handle_missing_vals(): imputation, missing values 
    - detect_outliers(): outliers
    """

    # Create message chain
    messages = []
    messages.append(SystemMessage(content=f"""
    You are a data cleaning agent
                                  
    Dataset info: Shape: {df.shape}, Sample: {df.head(3).to_string()}

    {features}

    Rules:
    - Each function takes (user_query, df) and returns modified df
    - Generate Python code that calls the functions with SPECIFIC instructions
    - Each function call should have a targeted query explaining exactly what to do
    - Return only executable Python code, no explanations, no markdown blocks
    - Only if no actions can be taken, print a descriptive message why

    Examples:
    - User: Find outliers for price and stock, Generated: Single: df = detect_outliers("find outliers in price, stock", df)
    - User: Impute numeric columns and generate mean and std for age, Generated: df = handle_missing_vals("impute numeric columns", df); df = get_summary("calculate mean and std for age", df)
    """))
    messages.append(HumanMessage(content=f"User request: {user_query}"))

    # Call LLM
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
    response = llm.invoke(messages)
    generated_code = response.content.strip()
    print(generated_code)

    # Execute AI generated code
    try:
        original_df = df.copy()
        exec(generated_code)
        return df
    except Exception as e:
        print(f"Error: {e}")
        print(f"Generated Code:{generated_code}")
        return original_df


In [8]:
# Sample data
df = pd.read_csv('sample_data/smoke.csv')
user_query="Generate some summary statistics"

In [7]:
main(user_query, df)

/var/folders/0p/s0mk2hxn1tg33lrmprln9gh80000gn/T/ipykernel_7532/1102505369.py:34: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")


df = handle_missing_vals("impute missing values", df); df = detect_outliers("find outliers in the dataset", df)
Error: name 'handle_missing_vals' is not defined
Generated Code:df = handle_missing_vals("impute missing values", df); df = detect_outliers("find outliers in the dataset", df)


,Year,State,Smoke everyday,Smoke some days,Former smoker,Never smoked
0,2010,AL,15.60%,6.30%,23.90%,54.20%
1,2010,AK,13.50%,6.80%,26.10%,53.60%
2,2010,AZ,10.70%,4.40%,27.90%,57.10%
3,2100,Arkansas,17.30%,5.60%,24.10%,53%
4,2010,California,7.50%,4.60%,23.10%,64.80%
...,...,...,...,...,...,...
871,1995,Virginia,18.70%,2.70%,25.20%,53.50%
872,1995,Washington,17.50%,2.40%,29.90%,50.20%
873,1995,West Virginia,23.70%,1.90%,23.30%,51.10%
874,1995,Wisconsin,18.20%,3.50%,27.60%,50.70%
